<a href="https://colab.research.google.com/github/hyungbinYun/sw_project/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: README.md               
  inflating: artificialNoAnomaly/artificialNoAnomaly/art_daily_no_noise.csv  
  inflating: artificialNoAnomaly/artificialNoAnomaly/art_daily_perfect_square_wave.csv  
  inflating: artificialNoAnomaly/artificialNoAnomaly/art_daily_small_noise.csv  
  inflating: artificialNoAnomaly/artificialNoAnomaly/art_flatline.csv  
  inflating: artificialNoAnomaly/artificialNoAnomaly/art_noisy.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_daily_flatmiddle.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_daily_jumpsdown.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_daily_jumpsup.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_daily_nojump.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_increase_spike_density.csv  
  inflating: artificialWithAnomaly/artificialWithAnomaly/art_load_balancer_spikes.csv  
  inflating: realAWSCloudwatch/realA

In [ ]:
# 데이터 조작과 분석, 시각화 등을 위한 파이썬 라이브러리와 모듈을 import하고 있다.
# pandas와 같은 파이썬 라이브러리에 대해 잘 모르고 있어서 코드를 가져온 후 하나씩 찾아봄
import math
import numpy as np
import pandas_datareader as web
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,RepeatVector, TimeDistributed
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.stats as stats
from sklearn.metrics import f1_score

In [ ]:
# 데이터 읽어온 후 시각화 과정
# 이상치를 직접 지정
# 데이터셋(dataset)에 대한 학습 데이터의 길이(training_data_len)를 계산(전체 데이터 셋의 80%).
csv_name='/content/realAdExchange/realAdExchange/exchange-4_cpc_results.csv'
df = pd.read_csv(csv_name)
df.dropna(inplace=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['value'], name='Value'))
fig.update_layout(showlegend=True, title=csv_name)
fig.show()

anomaly_points=[
        [
            "2014-04-10 16:15:00.000000",
            "2014-04-12 01:45:00.000000"
        ]
    ]
df['ranomaly'] = 0
for start, end in anomaly_points:
  df.loc[((df['timestamp'] >= start) & (df['timestamp'] <= end)), 'ranomaly'] = 1

prec=[]
data = df.filter(['value'])
dataset = data.values
training_data_len = math.ceil(0.8 * len(dataset))

In [ ]:
# 데이터 확인
# head()는 아무입력이 없을 시 기본값으로 5줄을 출력한다.
df.head()

,timestamp,value,ranomaly
0,2011-07-01 00:15:01,0.091795,0
1,2011-07-01 01:15:01,0.074414,0
2,2011-07-01 02:15:01,0.056984,0
3,2011-07-01 03:15:01,0.071225,0
4,2011-07-01 04:15:01,0.045466,0


In [ ]:
# 데이터 전처리 및 학습 데이터 생성
scaler = StandardScaler()
scaled_data = scaler.fit_transform(dataset)

train_data = scaled_data[0:training_data_len]
x_train=[]
y_train = []
Past=150
for i in range(Past,len(train_data)):
  x_train.append(train_data[i-Past:i,0])
  y_train.append(train_data[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
# 시계열 데이터를 각각 훈련 데이터와 테스트 데이터로 분리
test_data = scaled_data[training_data_len-Past: , : ]
x_test = []
y_test =  scaled_data[training_data_len : , : ]
for i in range(Past,len(test_data)):
  x_test.append(test_data[i-Past:i,0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
# LSTM 모델 생성 및 컴파일 과정

model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(x_train.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(x_train.shape[2])))
model.compile(optimizer='adam', loss='mean_squared_error')
fname = "weights/"+csv_name+'.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(fname, monitor="val_loss", mode="min",
save_best_only=True, verbose=0)
callbacks = [checkpoint]

In [ ]:
# 훈련 데이터를 이용하여 모델 학습
model.fit(x_train, y_train, batch_size=64, epochs=20,validation_split=0.1,
                callbacks=callbacks, shuffle=False, verbose=0)

In [ ]:
model.load_weights(fname)

In [ ]:
# LSTM모델을 이용하여 훈련 데이터에 관한 예측을 수행
x_train_pred = model.predict(x_train, verbose=0)
x_train_pred=np.reshape(x_train_pred,(x_train_pred.shape[0],x_train_pred.shape[1]))
# train_mae_loss = np.mean(np.abs(x_train_pred - x_train[:,0])/np.std(x_train_pred), axis=1)
train_mae_loss = (np.mean(np.abs(x_train_pred - x_train[:,0]), axis=1))

In [ ]:
# 테스트 데이터에 대한 예측 수행 후 실제값과 예측값의 평균절대오차(MAE)를 계산
threshold = 3*np.min(train_mae_loss)
x_test_pred = model.predict(x_test, verbose=0)
x_test_pred=np.reshape(x_test_pred,(x_test_pred.shape[0],x_test_pred.shape[1]))
test_mae_loss = np.mean(np.abs(x_test_pred-x_test[:,0]), axis=1)

In [ ]:
# 이상 탐지 수행
# test_mae_loss와 threshold의 값을 기준으로 이상값 판단
test_score_df = pd.DataFrame(df[training_data_len:])
test_score_df['loss'] = test_mae_loss
test_score_df['threshold'] = threshold
test_score_df['thresholdmax'] = threshold+0.5
test_score_df['anomaly'] = ((test_score_df['loss'] <=threshold))

for i in test_score_df.index:
    if test_score_df['anomaly'][i]:
      test_score_df['anomaly'][i-Past] = True
      test_score_df['anomaly'][i] = False

anomalies = test_score_df.loc[(test_score_df['anomaly'] == True)]
anomalies.shape
ranomal=test_score_df.loc[(test_score_df['ranomaly'] == True)]
# precision=np.count(np.where(test_score_df['ranomaly'] and test_score_df['anomaly']))/(np.count(np.where(test_score_df['ranomaly'] and test_score_df['anomaly']))+np.count(np.where(not test_score_df['ranomaly'] and test_score_df['anomaly'])))
score = f1_score(test_score_df['ranomaly'], test_score_df['anomaly'], average="binary")
prec.append(score)

<ipython-input-15-33389a3c8cac>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-33389a3c8cac>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-33389a3c8cac>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# plotly라이브러리를 통해 그래프를 생성하고 손실값과 임계값의 관계를 나타낸다.
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_score_df.index, y=test_score_df['loss'], name='Test loss'))
fig.add_trace(go.Scatter(x=test_score_df.index, y=test_score_df['threshold'], name='Threshold'))
#fig.add_trace(go.Scatter(x=test_score_df.index, y=test_score_df['thresholdmax'], name='Threshold'))

fig.update_layout(showlegend=True, title='Test loss vs. Threshold')
fig.show()

In [ ]:
# 앞 코드와 마찬가지로 plotly라이브러리를 이용하여 그래프를 생성하고 value와 Anomaly값을 보여준다.
fig_an = go.Figure()
fig_an.add_trace(go.Scatter(x=test_score_df.index, y=test_score_df['value'], name='Value'))
fig_an.add_trace(go.Scatter(x=anomalies.index, y=anomalies['value'], mode='markers', name='Anomaly'))
# fig_an.add_trace(go.Scatter(x=ranomal.index, y=ranomal['value'], mode='markers', name='Real_Anomaly'))
fig_an.update_layout(showlegend=True, title='Detected anomalies-'+csv_name)
fig_an.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['value'], name='Value'))
fig.update_layout(showlegend=True, title='')
fig.show()

In [ ]:
# 사실 저에게는 이번 첫번쨰 프로젝트는 뭘 했다기보다는 코드를 가져와 기본개념을 학습하는 과정이었다. 이번 개인+팀 프로젝트 기간이 길지 않지만 부족한만큼 좀 더 해서 뭐라도 챙겨갈 수 있는 기회가 됐으면한다